In [1]:
import random
import time
import math
# 랜덤한 좌표 생성 함수
def generate_random_coordinates(center_latitude, center_longitude, radius_meters, num_points):
    random_coordinates = []
    for _ in range(num_points):
        # 반경 내에서 랜덤하게 좌표 생성
        delta_lat = random.uniform(-1, 1) * radius_meters / 111300
        delta_lon = random.uniform(-1, 1) * (radius_meters / (111300 * 
                                                           math.cos(math.radians(center_latitude))))
        random_lat = center_latitude + delta_lat
        random_lon = center_longitude + delta_lon
        random_coordinates.append((random_lat, random_lon))
    return random_coordinates

# 반복 횟수 설정
num_iterations = 30

# 고정된 도착지 좌표 (카카오맵에서 확인 필요)
# 서울특별시 금천구 가산디지털2로 187
destination_lat = 37.4845817
destination_lon = 126.876185

# 반경 내 랜덤 좌표 생성
center_latitude = destination_lat
center_longitude = destination_lon
radius_meters = 600  # 600m 반경 내에서 랜덤 좌표 생성
num_points = 30
random_coordinates = generate_random_coordinates(center_latitude, center_longitude, radius_meters, num_points)

In [2]:
import folium

m = folium.Map(location=[destination_lat,destination_lon], zoom_start=14)
coordinates = random_coordinates

for coord in coordinates:
    folium.CircleMarker(location=coord, radius=5, color='blue', fill=True, fill_color='blue').add_to(m)

m

In [41]:
import time
from datetime import datetime
import os
os.environ['DISPLAY'] = ":0.0"
import pyautogui
import pyperclip
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

driver = webdriver.Chrome()
url = 'https://map.kakao.com/'
driver.get(url)
time.sleep(2.0)

# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait


key_word = '서울특별시 금천구 가산디지털2로 187'
# 고정된 도착지 좌표 (카카오맵에서 확인 필요)
# destination_lat = 37.4845817
# destination_lon = 126.876185

startlist = random_coordinates
endlist = ['서울특별시 금천구 가산디지털2로 187']*30
timelist0 = []
timelist1 = []
timelist2 = []
distancelist = []


for idx, (latitude, longitude) in enumerate(random_coordinates):
    driver.get(f"https://map.kakao.com/?q={latitude},{longitude}")
    print(idx,' 번째입니다..')
    time.sleep(3)  # 페이지가 로드되기를 기다림

    # css를 찾을때 까지 10초 대기
    time_wait(10, 'div.box_searchbar > input.query')
    
    # 출발지 설정
    print(latitude,',', longitude)
    search = driver.find_element(By.CSS_SELECTOR, 'div.box_searchbar > input.query')
    search.clear() # 기존 검색어 지우고
    time.sleep(2)
    search.send_keys(f"{latitude},{longitude}")
    search.send_keys(Keys.ENTER)
    time.sleep(2)

    # 버튼클릭
    find_route_button = driver.find_element(By.XPATH, '//*[@id="view.mapContainer"]/div[2]/div/div[6]/div[2]/div/div[2]/div[3]/div/div[2]/button')
    time.sleep(2)
    find_route_button.click()
    time.sleep(2)

    start_button = driver.find_element(By.XPATH, '//*[@id="view.mapContainer"]/div[2]/div/div[6]/div[2]/div/div[2]/div[3]/div/div[2]/div/button[1]')
    time.sleep(2)
    start_button.click()
    time.sleep(2)

    # 도착지 설정
    print(key_word)
    search = driver.find_element(By.XPATH, '//*[@id="info.route.waypointSuggest.input1"]')
    search.clear() # 기존 검색어 지우고
    time.sleep(2)
    search.send_keys(key_word)
    search.send_keys(Keys.ENTER)
    time.sleep(2)

    # 화면 불투명 막 제거
    pyautogui.moveTo(800, 400, )
    pyautogui.click(clicks=2, interval=1)

    transit = driver.find_element(By.XPATH, '//*[@id="cartab"]')
    transit.click()

    # 데이터 수집
    time.sleep(2)

    now = datetime.now()
    date = now.date()
    recordtime = now.time()
    howlong = driver.find_element(By.XPATH, '//*[@id="info.flagsearch"]/div[6]/ul/li/div[1]/div/div[1]/p/span[1]/span[1]').text 
    distance = driver.find_element(By.XPATH, '//*[@id="info.flagsearch"]/div[6]/ul/li/div[1]/div/div[1]/p/span[2]/span[1]').text # 수정;

    timelist0.append(date)
    timelist1.append(recordtime)
    timelist2.append(howlong)
    distancelist.append(distance)
        
import pandas as pd

dataframe = pd.DataFrame(None)
dataframe['출발'] = startlist
dataframe['도착'] = endlist
dataframe['측정일자'] = timelist0
dataframe['측정시간'] = timelist1
dataframe['걸린시간(분)'] = timelist2
dataframe['거리(km)'] = distancelist
dataframe = dataframe.astype({'걸린시간(분)':'int32', '거리(km)':'float'}) # float 실슈
dataframe['속도(km/h)'] = dataframe['거리(km)']/(dataframe['걸린시간(분)']/60)

dataframe.to_excel('./테스트용240319.xlsx')
dataframe


0  번째입니다..
37.48986296023662 , 126.87353217103838
서울특별시 금천구 가산디지털2로 187
1  번째입니다..
37.489085565661696 , 126.86984616893454
서울특별시 금천구 가산디지털2로 187
2  번째입니다..
37.48152334923718 , 126.88030085602604
서울특별시 금천구 가산디지털2로 187
3  번째입니다..
37.48024336054726 , 126.8751765914684
서울특별시 금천구 가산디지털2로 187
4  번째입니다..
37.48422564292012 , 126.86973585790507
서울특별시 금천구 가산디지털2로 187
5  번째입니다..
37.48415941294932 , 126.8823885775665
서울특별시 금천구 가산디지털2로 187
6  번째입니다..
37.48970179144432 , 126.87097443205786
서울특별시 금천구 가산디지털2로 187
7  번째입니다..
37.487819916786954 , 126.87047759408459
서울특별시 금천구 가산디지털2로 187
8  번째입니다..
37.48702385097668 , 126.86959593980811
서울특별시 금천구 가산디지털2로 187
9  번째입니다..
37.487117908933065 , 126.87554728031147
서울특별시 금천구 가산디지털2로 187
10  번째입니다..
37.4819271803251 , 126.87280142173428
서울특별시 금천구 가산디지털2로 187
11  번째입니다..
37.48846134662803 , 126.8748433043776
서울특별시 금천구 가산디지털2로 187
12  번째입니다..
37.48309822207354 , 126.87871496378338
서울특별시 금천구 가산디지털2로 187
13  번째입니다..
37.47974123205926 , 126.87724540290853
서울특별시 금천구 가

ValueError: invalid literal for int() with base 10: '1.2': Error while type casting for column '거리(km)'

In [45]:
dataframe = pd.DataFrame(None)
dataframe['출발'] = startlist
dataframe['도착'] = endlist
dataframe['측정일자'] = timelist0
dataframe['측정시간'] = timelist1
dataframe['걸린시간(분)'] = timelist2
dataframe['거리(km)'] = distancelist
dataframe = dataframe.astype({'걸린시간(분)':'int32', '거리(km)':'float'})
dataframe['속도(km/h)'] = dataframe['거리(km)']/(dataframe['걸린시간(분)']/60)
dataframe['속도(km/h)'] = dataframe['속도(km/h)'].round(2)

dataframe.to_excel('./테스트용240319.xlsx')
dataframe

,출발,도착,측정일자,측정시간,걸린시간(분),거리(km),속도(km/h)
0,"(37.48986296023662, 126.87353217103838)",서울특별시 금천구 가산디지털2로 187,2024-03-19,16:06:51.771362,4,1.2,18.00
1,"(37.489085565661696, 126.86984616893454)",서울특별시 금천구 가산디지털2로 187,2024-03-19,16:07:15.527997,9,3.4,22.67
2,"(37.48152334923718, 126.88030085602604)",서울특별시 금천구 가산디지털2로 187,2024-03-19,16:07:39.560728,7,1.9,16.29
3,"(37.48024336054726, 126.8751765914684)",서울특별시 금천구 가산디지털2로 187,2024-03-19,16:08:03.324146,4,0.9,13.50
4,"(37.48422564292012, 126.86973585790507)",서울특별시 금천구 가산디지털2로 187,2024-03-19,16:08:27.199797,5,0.8,9.60
5,"(37.48415941294932, 126.8823885775665)",서울특별시 금천구 가산디지털2로 187,2024-03-19,16:08:51.119844,6,1.8,18.00
6,"(37.48970179144432, 126.87097443205786)",서울특별시 금천구 가산디지털2로 187,2024-03-19,16:09:14.926226,19,9.2,29.05
7,"(37.487819916786954, 126.87047759408459)",서울특별시 금천구 가산디지털2로 187,2024-03-19,16:09:38.748691,5,0.9,10.80
8,"(37.48702385097668, 126.86959593980811)",서울특별시 금천구 가산디지털2로 187,2024-03-19,16:10:02.721514,5,0.9,10.80
9,"(37.487117908933065, 126.87554728031147)",서울특별시 금천구 가산디지털2로 187,2024-03-19,16:10:26.452710,2,0.3,9.00


오전 : 출발랜덤 + 도착 고정(주차장)
도착주소 : 서울특별시 금천구 가산디지털2로 187